In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
path = "../training/conf/network/encoder/efficientnet.yaml"

In [4]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [5]:
cfg

{'_target_': 'text_recognizer.networks.encoders.efficientnet.EfficientNet', 'arch': 'b0', 'stochastic_dropout_rate': 0.2, 'bn_momentum': 0.99, 'bn_eps': 0.001}

In [15]:
cfg.depth = 6

In [16]:
net = instantiate(cfg)

In [17]:
from torchinfo import summary

In [18]:
summary(net, (2, 1, 56, 1024), device="cpu", depth=4)

Layer (type:depth-idx)                        Output Shape              Param #
EfficientNet                                  --                        --
├─ModuleList: 1-1                             --                        --
├─Sequential: 1-2                             [2, 32, 28, 512]          --
│    └─ZeroPad2d: 2-1                         [2, 1, 57, 1025]          --
│    └─Conv2d: 2-2                            [2, 32, 28, 512]          288
│    └─BatchNorm2d: 2-3                       [2, 32, 28, 512]          64
│    └─Mish: 2-4                              [2, 32, 28, 512]          --
├─ModuleList: 1-1                             --                        --
│    └─MBConvBlock: 2-5                       [2, 16, 28, 512]          --
│    │    └─Depthwise: 3-1                    [2, 32, 28, 512]          --
│    │    │    └─Sequential: 4-1              [2, 32, 28, 512]          352
│    │    └─SqueezeAndExcite: 3-2             [2, 32, 28, 512]          --
│    │    │    └─S